In [2]:
import jupyterlab_dash
import dash
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import plotly.express as px

gapminder = px.data.gapminder().reset_index()

viewer = jupyterlab_dash.AppViewer()
app = dash.Dash(__name__)

app.layout = html.Div([
        html.H5("Lasso select scatter points to update the map"),
        dcc.Graph(
            id='main-figure',
            figure=px.scatter(gapminder, x="gdpPercap", y="lifeExp", 
                color="continent", size="pop", hover_data=['index'],
                hover_name="country", animation_frame="year", 
                log_x=True, size_max=60, height=300)
            ),
        dcc.Graph(
            id='other-figure',
            figure = px.scatter_geo(gapminder.query("year == 1952"), 
                locations="iso_alpha", 
                color="continent", hover_name="country", size="pop",
              projection="natural earth", height=500)
            ),
        ]
        )


@app.callback(
    Output('other-figure', 'figure'),
    [Input('main-figure', 'selectedData')])
def display_selected_data(selectedData):
    if selectedData:
        indices = [point["customdata"][0] for point in selectedData["points"]]
        df = gapminder.query("index in @indices")
        return px.scatter_geo(df, locations="iso_alpha", color="continent",
                size="pop",
                projection="natural earth", height=500)
    else:
        raise PreventUpdate


viewer.show(app)